In [ ]:
from tkinter import *
import tkinter
from tkinter import filedialog
import numpy as np
from tkinter.filedialog import askopenfilename
import pandas as pd 
from tkinter import simpledialog
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.utils.np_utils import to_categorical
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
from tensorflow.keras.models import load_model
import seaborn as sns
import os 
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam


main = tkinter.Tk()
main.title("Multivariate Gait Analysis for Healthy Subjects under Various Walking Conditions")
main.geometry("1000x650")

global filename
global fnn_model, mlp_model
global X_train, y_train, X_test, y_test

def upload():
    global filename
    global dataset
    filename = filedialog.askopenfilename(initialdir = "dataset")
    text.delete('1.0', END)
    text.insert(END,filename+' Loaded\n\n')
    dataset = pd.read_csv(filename)
    text.insert(END,str(dataset.head))

def preprocess():
    global dataset
    global X_train, y_train, X_test, y_test
    text.delete('1.0', END)
    X = dataset.drop("condition", axis=1).values
    Y = dataset['condition']
    le = LabelEncoder()
    Y = le.fit_transform(Y)
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=0)
    from sklearn.preprocessing import StandardScaler

    # Initialize the StandardScaler
    scaler = StandardScaler()

    # Fit the scaler on the training data and transform both the training and testing data
    X_train= scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)  # Use the same scaler instance to transform the test data

    text.insert(END,"\n\nTotal Records for training : "+str(len(X_train))+"\n")
    text.insert(END,str(X_train))
    text.insert(END,"\n"+str(y_train))
    # Assuming y_train is your original target variable
    y_train = to_categorical(y_train, num_classes=3)
    # Assuming y_test is your original test target variable
    y_test= to_categorical(y_test, num_classes=3)
def feedforwardneuralnetwork():
    global y_test
    if os.path.exists('model.h5'):
        # Load the existing model
        loaded_model = load_model('model.h5')
        # Evaluate the model
        loss, accuracy = loaded_model.evaluate(X_test, y_test)

        # Print the accuracy
        print(f'Test Accuracy: {accuracy * 100:.2f}%')
        # Generate classification report
        y_pred = loaded_model.predict(X_test)
        y_pred_classes = np.argmax(y_pred, axis=1)
        print(y_pred_classes)
        y_test1 = np.argmax(y_test, axis=1)
        Accuracy_score=accuracy_score(y_test1, y_pred_classes)
        text.insert(END, f"Neural Accuracy Score: { Accuracy_score}\n")
        report=classification_report(y_test1, y_pred_classes)
        text.insert(END, f"Neural Network classification_report: {report}\n")
        # Calculate confusion matrix
        cm = confusion_matrix(y_test1, y_pred_classes)
        text.insert(END, f"Neural Network confusion_matrix: {cm}\n")
        # Plot the confusion matrix as a heatmap
        plt.figure(figsize=(8, 6))
        sns.set(font_scale=1.2)
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
        plt.xlabel('Predicted')
        plt.ylabel('Actual')
        plt.title('Neural network Confusion Matrix')
        plt.show()

    else:
        
        from tensorflow.keras.models import Sequential
        from tensorflow.keras.layers import Dense, Dropout

        # Initialize the model
        model = Sequential()

        # Add layers to the model
        model.add(Dense(256, input_dim=X_train.shape[1], activation='relu'))
        model.add(Dropout(0.5))  # Adding dropout for regularization

        model.add(Dense(128, activation='relu'))
        model.add(Dropout(0.5))

        model.add(Dense(64, activation='relu'))
        model.add(Dropout(0.5))

        model.add(Dense(32, activation='relu'))
        model.add(Dropout(0.5))

        model.add(Dense(16, activation='selu'))
        model.add(Dropout(0.5))
        
        model.add(Dense(8, activation='selu'))
        model.add(Dropout(0.5))
        
        model.add(Dense(3, activation='softmax'))

        # Compile the model
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])  # Change the loss function to categorical_crossentropy

        early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

        # Train the model
        model.fit(X_train, y_train, epochs=150, batch_size=16, validation_data=(X_test, y_test), callbacks=[early_stopping])

        # Evaluate the model
        accuracy = model.evaluate(X_test, y_test)
        print(f'Test Accuracy:',accuracy)
        # Save FNN Model
        model.save('model.h5')  # Save the Keras model


def runMLP():
    global mlp_model
    global X_train, y_train, X_test, y_test
    mlp_model = MLPClassifier(hidden_layer_sizes=(32, 32), activation='relu', max_iter=100)
    mlp_model.fit(X_train, y_train)
    y_pred = mlp_model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    text.insert(END, f"Accuracy on Test Set (MLP): {accuracy}\n")
    report=classification_report(y_test, y_pred)
    text.insert(END, f"MLP classification_report: {report}\n")
    # Calculate confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    text.insert(END, f"MLP confusion_matrix: {cm}\n")
    # Plot the confusion matrix as a heatmap
    plt.figure(figsize=(8, 6))
    sns.set(font_scale=1.2)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('MLP Classifier Confusion Matrix')
    plt.show()

def close():
    main.destroy()

font = ('times', 15, 'bold')
title = Label(main, text='Multivariate Gait Analysis for Healthy Subjects under Various Walking Conditions', justify=LEFT)
title.config(bg='lavender blush', fg='DarkOrchid1')  
title.config(font=font)           
title.config(height=3, width=120)       
title.place(x=100,y=5)
title.pack()

font1 = ('times', 12, 'bold')
uploadButton = Button(main, text="Upload Multivariate Gait Dataset", command=upload)
uploadButton.place(x=10,y=100)
uploadButton.config(font=font1)  

preprocessButton = Button(main, text="Preprocess Dataset", command=preprocess)
preprocessButton.place(x=300,y=100)
preprocessButton.config(font=font1)

FNNButton = Button(main, text="Feed Forward Neural Network Algorithm", command=feedforwardneuralnetwork)
FNNButton.place(x=700,y=100)
FNNButton.config(font=font1)

MLPButton = Button(main, text="Run MLP Algorithm", command=runMLP)
MLPButton.place(x=480,y=100)
MLPButton.config(font=font1)

closeButton = Button(main, text="Close Application", command=close)
closeButton.place(x=10,y=200)
closeButton.config(font=font1)

font1 = ('times', 12, 'bold')
text=Text(main,height=20,width=160)
scroll=Scrollbar(text)
text.configure(yscrollcommand=scroll.set)
text.place(x=10,y=250)
text.config(font=font1) 

main.config(bg='light coral')
main.mainloop()


Train on 118561 samples, validate on 13174 samples
Epoch 1/150
118561/118561 [==============================] - 37s 308us/sample - loss: 1.0990 - acc: 0.3530 - val_loss: 1.0867 - val_acc: 0.3705
Epoch 2/150
118561/118561 [==============================] - 39s 331us/sample - loss: 1.0712 - acc: 0.3648 - val_loss: 1.0519 - val_acc: 0.3836
Epoch 3/150
118561/118561 [==============================] - 38s 324us/sample - loss: 1.0505 - acc: 0.3995 - val_loss: 1.0274 - val_acc: 0.4293
Epoch 4/150
118561/118561 [==============================] - 39s 328us/sample - loss: 1.0371 - acc: 0.4197 - val_loss: 1.0184 - val_acc: 0.4365
Epoch 5/150
118561/118561 [==============================] - 39s 331us/sample - loss: 1.0318 - acc: 0.4252 - val_loss: 1.0137 - val_acc: 0.4425
Epoch 6/150
118561/118561 [==============================] - 39s 328us/sample - loss: 1.0289 - acc: 0.4292 - val_loss: 1.0062 - val_acc: 0.4512
Epoch 7/150
118561/118561 [==============================] - 38s 319us/sample - loss: